In [1]:
import glob, os, json
import time
import pickle
import solver
import traceback

In [14]:
tspLibPath = "../data/tsplib/"

# Load TSP instances
tspLibInstances = []
tspInstanceNames = []
for file in glob.glob(tspLibPath + "*.tsp"):
    try:
        tsp = solver.loadTSPLib(file)
        tspLibInstances.append(tsp)
        tspInstanceNames.append(os.path.basename(file))
    except:
        traceback.print_exc()

In [15]:
generatedPath = "../data/generated/"

# Load TSP instances
generatedInstances = []
generatedInstanceNames = []
for file in glob.glob(generatedPath + "*.pytsp"):
    try:
        tspFile = open(file, "rb")
        tsp = pickle.load(tspFile)
        generatedInstances.append(tsp)
        generatedInstanceNames.append(os.path.basename(file))
        tspFile.close()
    except:
        traceback.print_exc()

In [16]:
allInstances = tspLibInstances + generatedInstances
allInstanceNames = tspInstanceNames + generatedInstanceNames

In [2]:
def measureFeature(name, deterministic, tsp, data, features, featureFunction):
    start = time.time()
    value = featureFunction(tsp, data)
    end = time.time()
    
    storedValue = value
    if type(storedValue).__module__=='numpy': # if value is numpy.*: > to python list
        storedValue = storedValue.tolist()
    
    if deterministic:
        features[name] = storedValue
    else:
        valueName = name + "Values"
        if valueName not in features:
            features[valueName] = []
            
        features[valueName].append(storedValue)
        
    timeName = name + "Times"
    
    if timeName not in features:
        features[timeName] = []
    
    features[timeName].append(end - start)
    
    return value

In [37]:
def stripNumpyDict(dictionary):
    for key in dictionary:
        value = dictionary[key]
        if isinstance(value, dict):
            stripNumpyDict(value)
        if isinstance(value, list):
            dictionary[key] = stripNumpyArray(value)
        elif type(value).__module__=='numpy': # if value is numpy.*: > to python list
            dictionary[key] = value.item()
            
def stripNumpyArray(array):
    newArray = []
    for item in array:
        if isinstance(item, list):
            newArray.append(stripNumpyArray(item))
        elif isinstance(item, numpy.ndarray):
            newArray.append(stripNumpyArray(item.tolist()))
        elif type(item).__module__=='numpy': # if value is numpy.*: > to python list
            newArray.append(item.item())
        else:
            newArray.append(item)
            
    return newArray

In [74]:
from network.features import *
from network.simpleFeatures import *
from network.complexFeatures import *
from network.heuristicFeatures import *
import json

nLowestEdgeCosts = 10
nGeneratedSolutions = 100

def generateSimpleFeatures(tsp, features, n):
    simple = {}
    for i in range(n):
        print("Starting Simple Features %d" % (i + 1))
        measureFeature("numberVertices", True, tsp, None, simple, lambda tsp, _: numberVertices(tsp))
        
        vertexCosts = measureFeature("vertexCostPrep", True, tsp, None, simple, lambda tsp, _: buildVertexCosts(tsp))
        data = {"vertexCosts": vertexCosts}
        measureFeature("lowestVertexCost", True, tsp, data, simple, lambda _, data: lowestVertexCost(data["vertexCosts"]))
        measureFeature("highestVertexCost", True, tsp, data, simple, lambda _, data: highestVertexCost(data["vertexCosts"]))
        measureFeature("averageVertexCost", True, tsp, data, simple, lambda _, data: averageVertexCost(data["vertexCosts"]))
        measureFeature("standardDeviationVertexCost", True, tsp, data, simple, lambda _, data: standardDeviationVertexCost(data["vertexCosts"]))
        measureFeature("medianVertexCost", True, tsp, data, simple, lambda _, data: medianVertexCost(data["vertexCosts"]))
        
        measureFeature("sumCostNearestNeighbor", True, tsp, None, simple, lambda tsp, _: sumCostNearestNeighbor(tsp))
        measureFeature("numberEdges", True, tsp, None, simple, lambda tsp, _: numberEdges(tsp))
        measureFeature("lowestEdgeCost", True, tsp, None, simple, lambda tsp, _: lowestEdgeCost(tsp))
        measureFeature("highestEdgeCost", True, tsp, None, simple, lambda tsp, _: highestEdgeCost(tsp))
        measureFeature("averageEdgeCost", True, tsp, None, simple, lambda tsp, _: averageEdgeCost(tsp))
        measureFeature("standardDeviationEdgeCost", True, tsp, None, simple, lambda tsp, _: standardDeviationEdgeCost(tsp))
        measureFeature("medianEdgeCost", True, tsp, None, simple, lambda tsp, _: medianEdgeCost(tsp))
        measureFeature("sumNLowestEdgeCost", True, tsp, None, simple, lambda tsp, _: sumNLowestEdgeCost(tsp, nLowestEdgeCosts))
        
        print("Ending Simple Features %d" % (i + 1))
        
    features["simpleFeatures"] = simple
    
def generateComplexFeatures(tsp, features, n):
    complexFeatures = {}
    for i in range(n):
        print("Starting Complex Features %d" % (i + 1))
        measureFeature("averageShortestPathGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: averageShortestPathGeodesicDistance(tsp))
        measureFeature("averageGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: averageGeodesicDistance(tsp))
        
        globalEfficiencyValue = measureFeature("globalEfficiency", True, tsp, None, complexFeatures, lambda tsp, _: globalEfficiency(tsp))
        data = {"globalEfficiency": globalEfficiencyValue}
        measureFeature("harmonicMeanGeodesicDistance", True, tsp, None, complexFeatures, lambda tsp, _: harmonicMeanGeodesicDistance(tsp))
        
        measureFeature("networkVulnerability", True, tsp, data, complexFeatures, lambda tsp, data: networkVulnerability(tsp, data["globalEfficiency"]))
        
        measureFeature("clusteringCoefficientTransitivity", True, tsp, None, complexFeatures, lambda tsp, _: clusteringCoefficientTransitivity(tsp))
        measureFeature("alternateClusteringCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: alternateClusteringCoefficient(tsp))
        
        connectedEdgeCountsValue = measureFeature("connectedEdgeCountsPrep", True, tsp, None, complexFeatures, lambda tsp, _: connectedEdgeCounts(tsp))
        data = {"connectedEdgeCounts": connectedEdgeCountsValue}
        measureFeature("weightedClusteringCoefficient", True, tsp, data, complexFeatures, lambda tsp, data: weightedClusteringCoefficient(tsp, data["connectedEdgeCounts"]))
        measureFeature("networkCyclicCoefficient", True, tsp, data, complexFeatures, lambda tsp, data: networkCyclicCoefficient(tsp, data["connectedEdgeCounts"]))
        measureFeature("maxVertexDegree", True, tsp, data, complexFeatures, lambda _, data: maxVertexDegree(data["connectedEdgeCounts"]))
        measureFeature("edgeDegreeCorrelation", True, tsp, data, complexFeatures, lambda tsp, data: edgeDegreeCorrelation(tsp, data["connectedEdgeCounts"]))
        
        measureFeature("entropyDegreeDistribution", True, tsp, None, complexFeatures, lambda tsp, _: entropyDegreeDistribution(tsp))
        measureFeature("targetEntropy", True, tsp, None, complexFeatures, lambda tsp, _: targetEntropy(tsp))
        measureFeature("vertexParticipationCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: vertexParticipationCoefficient(tsp))
        measureFeature("edgeReciprocity", True, tsp, None, complexFeatures, lambda tsp, _: edgeReciprocity(tsp))
        measureFeature("adjacencyCorrelationCoefficient", True, tsp, None, complexFeatures, lambda tsp, _: adjacencyCorrelationCoefficient(tsp))
        
        print("Ending Complex Features %d" % (i + 1))
        
    features["complexFeatures"] = complexFeatures
    
def generateHeuristicFeatures(tsp, features, n):
    heuristicFeatures = {}
    for i in range(n):
        print("Starting Heuristic Features %d" % (i + 1))
        
        random = measureFeature("randomSolutionsPrep", True, tsp, None, heuristicFeatures, lambda tsp, _: randomSolutions(tsp, nGeneratedSolutions))
        data = {"randomSolutions": random}
        randomNeighbor = measureFeature("randomNeighborSolutionsPrep", True, tsp, data, heuristicFeatures, lambda tsp, data: list(map(lambda solution: neighborSolutions(tsp, solution, nGeneratedSolutions), data["randomSolutions"])))
        
        # Remove most of randomNeighbors from features
        heuristicFeatures["randomNeighborSolutionsPrep"] = heuristicFeatures["randomNeighborSolutionsPrep"][:min(5, nGeneratedSolutions)]
        
        data = {"randomSolutions": random, "randomNeighborSolutions": randomNeighbor}
        measureFeature("proportionsNeighborsWithBetterSolution", True, tsp, data, heuristicFeatures, lambda tsp, data: proportionsNeighborsWithBetterSolution(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        measureFeature("averageRatioNeighborsWithBetterSolution", True, tsp, data, heuristicFeatures, lambda tsp, data: averageRatioNeighborsWithBetterSolution(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        measureFeature("randomNeighborSolutionQuality", True, tsp, data, heuristicFeatures, lambda tsp, data: randomNeighborSolutionQuality(tsp, data["randomSolutions"], data["randomNeighborSolutions"]))
        
        print("Ending Heuristic Features %d" % (i + 1))
        
    features["heuristicFeatures"] = heuristicFeatures
        
def generateFeatures(tsp, filename, n):
    features = {}
    
    generateSimpleFeatures(tsp, features, n)
    generateComplexFeatures(tsp, features, n)
    
    stripNumpyDict(features)
    
    with open(filename, "w") as file:
        json.dump(features, file, indent="\t")


In [41]:
allInstanceNames

NameError: name 'allInstanceNames' is not defined

In [53]:
tsp = solver.loadTSPLib("/Users/adam/Documents/Rose Repos/Thesis/data/tsplib/ch130.tsp")

In [75]:
features = {}
# generateFeatures(tsp, "../data/features/tsplib/ch130.json", 1)
generateHeuristicFeatures(tsp, features, 1)

Starting Heuristic Features 1
Ending Heuristic Features 1


In [55]:
stripNumpyDict(features)

with open("test.json", "w") as file:
    json.dump(features, file, indent="\t")

In [14]:
sortedCosts = numpy.sort(tsp.costs[numpy.nonzero(tsp.costs)], axis=None)

In [15]:
sortedCosts

array([  6.60180990e-01,   6.60180990e-01,   2.23623041e+00, ...,
         9.38517244e+02,   9.38842335e+02,   9.38842335e+02])

In [16]:
tsp.costs

array([[   0.        ,  119.09645956,  169.64633859, ...,   89.76011102,
         166.56933504,   81.6422314 ],
       [ 119.09645956,    0.        ,  139.19177174, ...,  175.81408796,
         270.55303015,   57.19837622],
       [ 169.64633859,  139.19177174,    0.        , ...,  146.43591543,
         332.82245704,  105.86886018],
       ..., 
       [  89.76011102,  175.81408796,  146.43591543, ...,    0.        ,
         207.5325414 ,  119.84887942],
       [ 166.56933504,  270.55303015,  332.82245704, ...,  207.5325414 ,
           0.        ,  246.87507296],
       [  81.6422314 ,   57.19837622,  105.86886018, ...,  119.84887942,
         246.87507296,    0.        ]])

In [17]:
tsp.costs[numpy.nonzero(tsp.costs)]

array([ 119.09645956,  169.64633859,  248.29638486, ...,  207.21526864,
        119.84887942,  246.87507296])